In [1]:
import pandas as pd
import numpy as np

In [3]:
# 3. Tải dữ liệu và in ra 10 dòng đầu/cuối
df = pd.read_csv('dulieuxettuyendaihoc.csv')

print("10 dòng đầu tiên:")
print(df.head(10))
print("\n10 dòng cuối cùng:")
print(df.tail(10))

10 dòng đầu tiên:
   STT   T1   L1   H1   S1   V1   X1   D1   N1   T2  ...   X6   D6   N6  GT  \
0    1  7.2  7.3  6.3  7.3  7.0  7.9  7.3  5.5  8.4  ...  6.6  7.6  5.9   F   
1    2  5.4  3.9  3.9  4.0  5.4  5.4  5.3  2.8  6.3  ...  6.6  6.1  4.4   M   
2    3  5.6  6.8  7.2  7.5  4.3  7.4  5.8  3.2  5.0  ...  7.9  8.1  4.6   M   
3    4  6.6  6.4  5.3  6.9  5.4  7.3  6.4  5.8  5.1  ...  7.1  7.3  7.4   M   
4    5  6.0  5.0  6.0  7.3  6.5  7.7  7.9  6.1  5.4  ...  6.1  7.5  7.2   M   
5    6  9.3  7.6  7.9  8.6  7.0  7.3  7.7  7.9  9.6  ...  5.7  8.0  7.8   M   
6    7  2.8  3.9  5.5  6.9  5.0  7.3  4.6  5.2  4.4  ...  6.6  6.0  6.0   F   
7    8  8.3  6.0  7.6  5.1  7.5  4.7  5.8  7.2  6.7  ...  7.1  6.8  7.0   F   
8    9  6.5  6.3  7.6  6.0  5.5  7.1  6.3  5.0  7.3  ...  9.1  7.9  6.1   F   
9   10  7.3  5.9  4.7  7.1  6.7  7.9  6.7  7.7  8.0  ...  6.4  6.1  7.8   F   

   DT   KV   DH1   DH2   DH3  KT  
0 NaN  2NT  3.25  3.25  4.50  A1  
1 NaN    1  6.00  4.00  3.50   C  
2 NaN  

In [4]:
# 4. Thống kê và xử lý dữ liệu thiếu cho cột 'DT' (Dân tộc)
print("\nSố lượng dữ liệu thiếu cột DT trước khi xử lý:", df['DT'].isnull().sum())
# Thay thế dữ liệu thiếu bằng 0
df['DT'].fillna(0, inplace=True)


Số lượng dữ liệu thiếu cột DT trước khi xử lý: 97


/tmp/ipython-input-4282496904.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DT'].fillna(0, inplace=True)


In [5]:
# 5. Thống kê và xử lý dữ liệu thiếu cho cột 'T1'
print("Số lượng dữ liệu thiếu cột T1 trước khi xử lý:", df['T1'].isnull().sum())
# Thay thế bằng Mean
df['T1'].fillna(df['T1'].mean(), inplace=True)

Số lượng dữ liệu thiếu cột T1 trước khi xử lý: 0


/tmp/ipython-input-2230593478.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['T1'].fillna(df['T1'].mean(), inplace=True)


In [6]:
# 6. Xử lý dữ liệu thiếu cho các biến điểm số còn lại (T1->N6, DH1->DH3)
# Tạo danh sách các cột điểm số cần xử lý
score_cols = [c for c in df.columns if df[c].dtype in ['float64', 'int64'] and c not in ['STT', 'DT', 'GT', 'KV', 'KT']]

for col in score_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mean(), inplace=True)

In [7]:
# 7. Tạo biến TBM1, TBM2, TBM3 (Lớp 10, 11, 12)
# Lưu ý: Theo đề bài, T1=Lớp 10, T2=Lớp 11, T6=Lớp 12
def cal_tbm(row, suffix):
    try:
        # Công thức: (T*2 + L + H + S + V*2 + X + D + N) / 10
        t = row['T' + suffix]
        l = row['L' + suffix]
        h = row['H' + suffix]
        s = row['S' + suffix]
        v = row['V' + suffix]
        x = row['X' + suffix]
        d = row['D' + suffix]
        n = row['N' + suffix]
        return (t*2 + l + h + s + v*2 + x + d + n) / 10
    except KeyError:
        return np.nan

df['TBM1'] = df.apply(lambda r: cal_tbm(r, '1'), axis=1) # Lớp 10
df['TBM2'] = df.apply(lambda r: cal_tbm(r, '2'), axis=1) # Lớp 11
df['TBM3'] = df.apply(lambda r: cal_tbm(r, '6'), axis=1) # Lớp 12

In [8]:
# 8. Tạo biến Xếp loại (XL1, XL2, XL3)
def xep_loai(score):
    if score < 5.0: return 'Y'
    elif 5.0 <= score < 6.5: return 'TB'
    elif 6.5 <= score < 8.0: return 'K'
    elif 8.0 <= score < 9.0: return 'G'
    else: return 'XS' # >= 9.0

df['XL1'] = df['TBM1'].apply(xep_loai)
df['XL2'] = df['TBM2'].apply(xep_loai)
df['XL3'] = df['TBM3'].apply(xep_loai)

In [9]:
# 9. Chuyển đổi thang điểm 10 sang 4 (Min-Max Normalization)
# Công thức Min-Max từ thang [0,10] sang [0,4]: New = (Old - 0) / (10 - 0) * 4
df['US_TBM1'] = (df['TBM1'] / 10) * 4
df['US_TBM2'] = (df['TBM2'] / 10) * 4
df['US_TBM3'] = (df['TBM3'] / 10) * 4

In [10]:
# 10. Tạo biến KQXT (Đậu/Rớt)
def ket_qua_xet_tuyen(row):
    kt = row['KT']
    dh1, dh2, dh3 = row['DH1'], row['DH2'], row['DH3']
    avg = 0

    if kt in ['A', 'A1']:
        avg = (dh1*2 + dh2 + dh3) / 4
    elif kt == 'B':
        avg = (dh1 + dh2*2 + dh3) / 4
    else:
        avg = (dh1 + dh2 + dh3) / 3

    return 1 if avg >= 5.0 else 0

df['KQXT'] = df.apply(ket_qua_xet_tuyen, axis=1)

In [11]:
# 11. Lưu file
df.to_csv('processed_dulieuxettuyendaihoc.csv', index=False)
print("Đã xử lý xong và lưu file 'processed_dulieuxettuyendaihoc.csv'.")

Đã xử lý xong và lưu file 'processed_dulieuxettuyendaihoc.csv'.
